In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from astropy.coordinates import Distance
from astropy import units as u
from astropy.coordinates import SkyCoord
from mpl_toolkits.mplot3d import Axes3D
from astropy.io import fits

plt.style.use('seaborn-dark-palette')
plt.style.use('ggplot')
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

%matplotlib inline

In [4]:
def loadSDSS():
    z=[]
    with open('./data/Skyserver_SQL_1_xyz.csv','r') as f:
        for line in f:
            if line[0]!='#' and len(line)>5:
                tmp = line.split(',')
                z.append(float(tmp[1]))
    with open('./data/Skyserver_SQL_2_xyz.csv','r') as f:
        for line in f:
            if line[0]!='#' and len(line)>5:
                tmp = line.split(',')
                z.append(float(tmp[1]))
    z = np.array(z,'d')
    
    return z

def loadUVES():
    # UVES Sky background files (continuum and emission lines)
    filename4 = homedir+'/Dropbox/UVES_Sky_Background/UVES_sky_all/fluxed_sky_437.fits'
    filename3 = homedir+'/Dropbox/UVES_Sky_Background/UVES_sky_all/fluxed_sky_580L.fits'
    filename1 = homedir+'/Dropbox/UVES_Sky_Background/UVES_sky_all/fluxed_sky_580U.fits'
    filename2 = homedir+'/Dropbox/UVES_Sky_Background/UVES_sky_all/fluxed_sky_860L.fits'
    hdulist1 = fits.open(filename1)
    hdulist2 = fits.open(filename2)
    hdulist3 = fits.open(filename3)
    hdulist4 = fits.open(filename4)

    flux1 = hdulist1[0].data
    flux2 = hdulist2[0].data
    flux3 = hdulist3[0].data
    flux4 = hdulist4[0].data
    
    CRPIX1 = hdulist1[0].header['CRPIX1']
    CRVAL1 = hdulist1[0].header['CRVAL1']
    CDELT1 = hdulist1[0].header['CDELT1']
    #print(CRPIX1,CRVAL1,CDELT1)
    hdulist1.close()
    begin = CRVAL1 - CRPIX1*CDELT1   
    end = begin + len(flux1)*CDELT1
    wavelength1 = np.arange(begin,end,CDELT1)
    #print(len(wavelength1),len(flux1))

    CRPIX1 = hdulist2[0].header['CRPIX1']
    CRVAL1 = hdulist2[0].header['CRVAL1']
    CDELT1 = hdulist2[0].header['CDELT1']
    #print(CRPIX1,CRVAL1,CDELT1)
    hdulist2.close()
    begin = CRVAL1 - CRPIX1*CDELT1   
    end = begin + len(flux2)*CDELT1
    wavelength2 = np.arange(begin,end,CDELT1)
    #print(len(wavelength2),len(flux2))

    CRPIX1 = hdulist3[0].header['CRPIX1']
    CRVAL1 = hdulist3[0].header['CRVAL1']
    CDELT1 = hdulist3[0].header['CDELT1']
    #print(CRPIX1,CRVAL1,CDELT1)
    hdulist3.close()
    begin = CRVAL1 - CRPIX1*CDELT1   
    end = begin + len(flux3)*CDELT1
    wavelength3 = np.arange(begin,end,CDELT1)
    #print(len(wavelength2),len(flux2))
    
    CRPIX1 = hdulist4[0].header['CRPIX1']
    CRVAL1 = hdulist4[0].header['CRVAL1']
    CDELT1 = hdulist4[0].header['CDELT1']
    #print(CRPIX1,CRVAL1,CDELT1)
    hdulist4.close()
    begin = CRVAL1 - CRPIX1*CDELT1   
    end = begin + len(flux4)*CDELT1
    wavelength4 = np.arange(begin,end,CDELT1)
    #print(len(wavelength2),len(flux2))
    
    return flux1,flux2,wavelength1,wavelength2,flux3,wavelength3,flux4,wavelength4

def loadUVESlinelists():
    # UVES identified line lists
    line_wavelength = []
    with open(homedir+'/Dropbox/UVES_Sky_Background/UVES_ident/gident_437.dat','r') as f:
        for line in f:
            if line[0]!='#' and len(line)>5:
                tmp = line.split()
                line_wavelength.append(tmp[1])
    with open(homedir+'/Dropbox/UVES_Sky_Background/UVES_ident/gident_580L.dat','r') as f:
        for line in f:
            if line[0]!='#' and len(line)>5:
                tmp = line.split()
                line_wavelength.append(tmp[1])
    with open(homedir+'/Dropbox/UVES_Sky_Background/UVES_ident/gident_580U.dat','r') as f:
        for line in f:
            if line[0]!='#' and len(line)>5:
                tmp = line.split()
                line_wavelength.append(tmp[1])
    with open(homedir+'/Dropbox/UVES_Sky_Background/UVES_ident/gident_860L.dat','r') as f:
        for line in f:
            if line[0]!='#' and len(line)>5:
                tmp = line.split()
                line_wavelength.append(tmp[1])
    line_wavelength = np.array(line_wavelength,'d')
    
    return line_wavelength


In [5]:
def plotskybackground2(ax,start=4500.,end=5550.):
    indices3 = (wavelength3>start) & (wavelength3<end)
    indices4 = (wavelength4>start) & (wavelength4<end)
    indices1 = (wavelength1>start) & (wavelength1<end)
    print len(indices3)
    print len(indices4)
    if len(wavelength1[indices1])>0:
        newstart = wavelength1[indices1][-1]
        print newstart
    else:
        newstart=start
        print newstart
    indices2 = (wavelength2>newstart) & (wavelength2<end)
    ax.plot(wavelength1[indices1],flux1[indices1],color='gray',zorder=2)
    ax.plot(wavelength2[indices2],flux2[indices2],color='gray',zorder=2)
    ax.plot(wavelength3[indices3],flux3[indices3],color='gray',zorder=2)
    ax.plot(wavelength4[indices4],flux4[indices4],color='gray',zorder=2)
    
    ## plot sky background divided by 10 to pick out strong lines
    ax.plot(wavelength1[indices1],flux1[indices1]/10.,color='red',zorder=2)
    ax.plot(wavelength2[indices2],flux2[indices2]/10.,color='red',zorder=2)
    ax.plot(wavelength3[indices3],flux3[indices3]/10.,color='red',zorder=2)
    ax.plot(wavelength4[indices4],flux4[indices4]/10.,color='red',zorder=2)
#    ax.plot(np.concatenate([wavelength1[indices1],wavelength2[indices2]]),
#            np.concatenate([flux1[indices1],flux2[indices2]]))
    ax.set_ylim(0,2)
    ax.set_xlim(start,end)
    ax.set_ylabel('Flux (sky)')
    
 #   # identified lines
 #   for line in line_wavelength:
 #       ax.plot([line],[1.5],'ro')
 #   
 #   # locations off 50Mpc, 100Mpc, 200Mpc, 500Mpc away
 #   for wavelen,name in zip([lambda_50,lambda_100,lambda_200,lambda_500],['50 Mpc','100 Mpc','200 Mpc','500 Mpc']):
 #       ax.plot([wavelen,wavelen],[0.,3.],'k--',linewidth=2)
 #      # if (wavelen>lowlim) and (wavelen<(lowlim+300.)):
 #      #     plt.text(line,1.5,name,rotation=90)
 #       
 #   # pretend filter (3 nm wide)
 #   #ax.fill_between([lowlim+240.,lowlim+240.+30.], 0, 1.5,facecolor ='yellow',alpha=0.4)
 #   #ax.plot([lowlim+240.,lowlim+240.,lowlim+240.+30.,lowlim+240.+30.,lowlim+240.],[0,1.5,1.5,0,0],'g-',linewidth=2)

    ax.set_xlabel('wavelength (Angstrom)')

In [6]:
machine = 'coho'
if machine=='chinook':
    homedir = 'Users/lokhorst'
elif machine=='coho':
    homedir = '/Users/deblokhorst'

In [7]:
z=loadSDSS()
flux1,flux2,wavelength1,wavelength2,flux3,wavelength3,flux4,wavelength4=loadUVES()
line_wavelength=loadUVESlinelists()

IOError: [Errno 2] No such file or directory: './data/Skyserver_SQL_1_xyz.csv'